##### Copyright 2021 The Cirq Developers

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Calibrations
The [tutorial on calibration](https://quantumai.google/cirq/google/calibration) gives an introduction to calibration
metrics and how to retrieve them from the Google Quantum Computing Service (QCS). In this tutorial, we show to visualize these calibration metrics
via single qubit heatmaps and two qubit interaction heatmaps.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/tutorials/google/visualizing_calibration_metrics>"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/tutorials/google/visualizing_calibration_metrics.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/tutorials/google/visualizing_calibration_metrics.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/tutorials/google/visualizing_calibration_metrics.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

import os
from random import random
from itertools import combinations

## Authenticate and install Cirq

For details of authentication and installation, please see [go/quantum-engine-quickstart](http://go/quantum-engine-quickstart).

Note:  The below code will install the latest stable release of cirq.  If you need the latest and greatest features and don't mind if a few things aren't quite working correctly, you can install the pre-release version of `cirq` using `pip install --pre cirq` instead of `pip install cirq` to get the most up-to-date features of cirq.

1. Enter the Cloud project ID you'd like to use in the `project_id` field.
2. Then run the cell below (and go through the auth flow for access to the project id you entered).

<img src="https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/images/run-code-block.png" alt="Quantum Engine console">

In [ ]:
# The Google Cloud Project id to use.
project_id = '' #@param {type:"string"}

if project_id == 'YOUR_PROJECT_ID':
    if 'GOOGLE_CLOUD_PROJECT' not in os.environ:
        raise Exception("Please setup project_id in this cell or set the `GOOGLE_CLOUD_PROJECT` env var to your project id.")
    project_id = os.environ['GOOGLE_CLOUD_PROJECT']
else:
    os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

def authenticate_user():
  """Runs the user through the Colab OAuth process.

  Checks for Google Application Default Credentials and runs interactive login
  if the notebook is executed in Colab. In case the notebook is executed in Jupyter notebook
  or other IPython runtimes, no interactive login is provided, it is assumed that the
  `GOOGLE_APPLICATION_CREDENTIALS` env var is set or `gcloud auth application-default login`
  was executed already.

  For more information on using Application Default Credentials see
  https://cloud.google.com/docs/authentication/production
  """
  in_colab = False
  try:
     from IPython import get_ipython
     in_colab = 'google.colab' in str(get_ipython())
  except:
     # Notebook is not executed within IPython. Assuming external authentication.
     return

  if in_colab:
      from google.colab import auth
      print("Getting OAuth2 credentials.")
      print("Press enter after entering the verification code.")
      auth.authenticate_user(clear_output=False)
      print("Authentication complete.")
  else:
      print("Notebook is not executed with Colab, assuming Application Default Credentials are setup.")

authenticate_user()

print("Successful authentication to Google Cloud.")

## Heatmaps for Calibration Metrics

Calibration Metrics about the performance of the quantum processor can be retrieved from the Google QCS.

In [ ]:
# Uncomment and replace PROCESSOR_ID.
# cals = cirq.google.get_engine_calibration(PROCESSOR_ID, project_id)

As an example, we create the calibration object with dummy data as follows:


In [ ]:
metrics = {'single_qubit_errors' : {}, 'two_qubit_errors': {}}
for qubit in cirq.google.Sycamore.qubits:
    metrics['single_qubit_errors'].update({(qubit,) : [random()]})
for q_1, q_2 in combinations(cirq.google.Sycamore.qubits, 2):
    if q_1.is_adjacent(q_2):
        metrics['two_qubit_errors'].update({(q_1, q_2) : [random()]})
cals = cirq.google.Calibration(metrics=metrics)

You can plot the single qubit heatmaps by fetching the single qubit metrics from calibration data as follows:

In [ ]:
cals.heatmap('single_qubit_errors').plot()

Similarly, you can fetch two-qubit metrics by passing the appropriate key, and plot the two qubit interaction heatmaps.

You can also change various properties of the plot by passing appropriate `kwargs` to the `update_config` method as shown below

In [ ]:
two_qubit_heatmap = cals.heatmap('two_qubit_errors')
two_qubit_heatmap.update_config(annotation_format='.2f')
two_qubit_heatmap.plot()